<a href="https://colab.research.google.com/github/Valerie-Fan/painting-classification/blob/main/painting_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-306-gb599ae42 Python-3.10.12 torch-2.2.0+cpu CPU


Setup complete ✅ (96 CPUs, 334.6 GB RAM, 15.7/225.3 GB disk)


## Preparing Dataset

In [ ]:
# ensure we're in the right directory to download our custom dataset
import os
os.makedirs("../datasets/", exist_ok=True)
%cd ../datasets/

/content/datasets


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="ISPQWm6pzaN1orqr2nuk")
project = rf.workspace("wikiart-v4").project("wikiart-v4")
version = project.version(1)
dataset = version.download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to wikiart-v4-1 in folder:: 100%|██████████| 3964/3964 [00:00<00:00, 5785.11it/s]


In [ ]:
# save the dataset name to the environment so we can use it in a system call later
dataset_name = dataset.location.split(os.sep)[-1]
os.environ["DATASET_NAME"] = dataset_name

## Training

In [ ]:
%cd ../yolov5
!python classify/train.py --model yolov5s-cls.pt --data $DATASET_NAME --epochs 100 --img 128 --pretrained weights/yolov5s-cls.pt

/content/yolov5
/usr/local/lib/python3.10/dist-packages/jax/_src/cloud_tpu_init.py:73: UserWarning: JAX_USE_PJRT_C_API_ON_TPU no longer has an effect (the new TPU runtime is always enabled now). Unset the environment variable to disable this warning.
  warnings.warn(
classify/train: model=yolov5s-cls.pt, data=wikiart-v4-1, epochs=100, batch_size=64, imgsz=128, nosave=False, cache=None, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=weights/yolov5s-cls.pt, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-306-gb599ae42 Python-3.10.12 torch-2.2.0+cpu CPU

TensorBoard: Start with 'tensorboard --logdir runs/train-cls', view at http://localhost:6006/
albumentations: ⚠️ not found, install with `pip install albumentations` (recommended)
100% 10.5M/10.5M [00:00<00:00, 118MB/s]

Model summary: 149 layers, 41878

In [ ]:
# get the path of an image from the test or validation set
if os.path.exists(os.path.join(dataset.location, "test")):
  split_path = os.path.join(dataset.location, "test")
else:
  os.path.join(dataset.location, "val")
example_class = os.listdir(split_path)[0]
example_image_name = '/content/datasets/wikiart-v4-1/test/abstract/ad-reinhardt_number-107-1950_jpg.rf.fb1e8a63dd18ad5264bb2b2514635171.jpg'
example_image_path = os.path.join(split_path, example_class, example_image_name)
os.environ["TEST_IMAGE_PATH"] = example_image_path

print(f"Inferring on an example of the class '{example_class}'")

#Infer
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source $TEST_IMAGE_PATH

Inferring on an example of the class 'romanticism'
/usr/local/lib/python3.10/dist-packages/jax/_src/cloud_tpu_init.py:73: UserWarning: JAX_USE_PJRT_C_API_ON_TPU no longer has an effect (the new TPU runtime is always enabled now). Unset the environment variable to disable this warning.
  warnings.warn(
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/datasets/wikiart-v4-1/test/abstract/ad-reinhardt_number-107-1950_jpg.rf.fb1e8a63dd18ad5264bb2b2514635171.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-306-gb599ae42 Python-3.10.12 torch-2.2.0+cpu CPU

Fusing layers... 
Model summary: 117 layers, 4182060 parameters, 0 gradients, 10.4 GFLOPs
image 1/1 /content/datasets/wikiart-v4-1/test/abstract/ad-reinhardt_number-107-1950_jpg.rf.fb1e8a63dd18ad5264bb2b2514635